In [1]:
import pandas as pd
import openai
import os
from annotation_utils import get_val_scores, make_transcript
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler

openai.api_key_path = "/home/alex/.personal/openAIkey"  # read protected to my 174 account

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# val_scores_df = get_val_scores()
# val_df = pd.read_json("data/label-validation/transcripts_combined.json")
# val_df.index = val_df["message_id"]
# val_df = val_df.join(val_scores_df, how="inner")
# main_df = val_df

# apt_tagger = AptTagger(
#     model_name="data/apt-tagger-deberta-xlarge-mnli-s63day9p",
#     device="cuda"
# )

# main_df["tag_mask"] = apt_tagger.tag(main_df["assistant_text"])
# main_df["annotated_assistant_text"] = main_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
# main_df["annotated_transcript"] = main_df.apply(make_transcript, axis=1, args=(True,))
# main_df.head()

In [3]:
annotated_val_df = pd.read_json("data/label-validation/annotated_transcripts_combined.json")
main_df = annotated_val_df

In [4]:
from Retriever import Retriever
retriever = Retriever(
    use_IVF=True,
    encoder_name="BAAI/bge-small-en",
    pretrained_index_path='retrieval/20230601.en.wiki_embeddings_bge-small-en_IVF.index',
)

Loading model... done.
Loading pretrained index... done.


In [46]:
truth_labeler = TruthLabeler(
                retriever,
                model_name="gpt-4",
                temperature=1,
                n_samples=5,
                uncertainty_bias=0,
                na_bias=0,
)

In [53]:
all_out_df = truth_labeler.label(
    ids=main_df["message_id"],
    annotated_transcripts=main_df["annotated_transcript"], 
    retrieval_queries=main_df["assistant_text"],
    n_threads=1)

Total cost: $0.8525
SKIPPING: scores must be one of ['N/A', 'Determinable - Certainly True', 'Determinable - True', 'Determinable - Uncertain', 'Determinable - False', 'Determinable - Certainly False'], but found ['Determinable - Uncertain,', 'Determinable - Uncertain,', 'Determinable - Uncertain,', 'Determinable - Uncertain,', 'Determinable - Uncertain,', 'Determinable - Uncertain,', 'Determinable - Uncertain,', 'Determinable - Uncertain,']
Total cost: $0.9432
Total cost: $1.0454
SKIPPING: scores must be one of ['N/A', 'Determinable - Certainly True', 'Determinable - True', 'Determinable - Uncertain', 'Determinable - False', 'Determinable - Certainly False'], but found ["Given that the information provided in this transcript is part of a fictional story, it's not possible to cross-check the accuracy of every detail from external sources or with context from a previous text. However, it's possible to judge if the stated plot developments and character actions are plausible or consisten

In [54]:
# NOTE: THERE ARE DUPLICATE IDS IN THE OUTPUT DF WHEN USING RETRIEVER SO THAT WE CAN COMPARE THE RESULTS
# TODO: Change this once I've finished validating the retriever
out_df_retr = all_out_df[all_out_df["used_retrieval"]]
retr_ids = set(out_df_retr["message_id"])
out_df_counter_retr = all_out_df[all_out_df["message_id"].isin(retr_ids) & ~all_out_df["used_retrieval"]]  # counterfactually if retrieval was not used
# remove all the rows with message_id in retr_ids but with used_retrieval==False
out_df = all_out_df[~(all_out_df["message_id"].isin(retr_ids) & ~all_out_df["used_retrieval"])]

In [70]:
main_df.index = main_df["message_id"]
labeled_df = out_df.copy()
labeled_df.index = labeled_df["message_id"]
labeled_df = labeled_df.drop(columns=["message_id", "annotated_transcript"])
labeled_df = labeled_df.join(main_df, how="left")
print("length:", len(labeled_df))
labeled_df.head()

length: 21


,input,retrieval_query,responses,prompt_tokens,completion_tokens,dollars,pred_scores,p_apts,p_trues,ann_count,...,model,assistant_text,user_text,message_id,prev_messages,rank,scores,tag_mask,annotated_assistant_text,annotated_transcript
message_id,,,,,,,,,,,,,,,,,,,,,
87cdf227-c048-4ccb-a790-0f44dc5ebb6b,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"As an artificial intelligence language model, ...",[[[1]] Score: Determinable - Certainly True\n[...,521,715,0.05853,"[(Determinable - Certainly True, N/A, Determin...","[0.6, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 1.0, ...","[0.8, 0.9199999999999999, 0.96, 0.96, 0.96, 0....",13,...,keyfan/bloomz-rlhf(max_length=512;temperature=...,"As an artificial intelligence language model, ...",Where was your last vaccation?,87cdf227-c048-4ccb-a790-0f44dc5ebb6b,[Where was your last vaccation?],NaN,"[Certainly True, Certainly True, Certainly Tru...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","As an artificial intelligence language model, ...",USER: Where was your last vaccation?\n\nASSIST...
fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,Joseph Scaliger (1557-1636) was an Italian sch...,[[[1]] Score: Determinable - Uncertain\n[[2]] ...,560,475,0.04530,"[(Determinable - Uncertain, Determinable - Unc...","[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",8,...,keyfan/bloomz-rlhf(max_length=512;temperature=...,Joseph Scaliger (1557-1636) was an Italian sch...,Describe Joseph Scaliger and his contributions...,fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,[Describe Joseph Scaliger and his contribution...,NaN,"[Certainly False, True, True, Certainly True, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Joseph Scaliger (1557-1636) was an Italian sch...,USER: Describe Joseph Scaliger and his contrib...
69f27058-6253-42bc-892c-5f8a95870ec1,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,Certainly! A well-chosen color palette is one ...,[[[1]] Score: Determinable - True\n[[2]] Score...,917,1246,0.10227,"[(Determinable - True, Determinable - Certainl...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9199999999999999, 0.9199999999999999, 0.880...",23,...,keyfan/bloomz-rlhf(max_length=512;temperature=...,Certainly! A well-chosen color palette is one ...,"I am designing a website, can you give me a co...",69f27058-6253-42bc-892c-5f8a95870ec1,"[I am designing a website, can you give me a c...",NaN,"[N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! A well-chosen color palette is one ...,"USER: I am designing a website, can you give m..."
b864688c-62cb-420d-bae1-e81677b4fded,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,Lena was a happy-go-lucky suburban housewife w...,[[[1]] Score: N/A\n[[2]] Score: N/A\n[[3]] Sco...,906,544,0.05982,"[(N/A, N/A), (N/A, N/A), (N/A, N/A), (N/A, N/A...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...",21,...,keyfan/bloomz-rlhf(max_length=512;temperature=...,Lena was a happy-go-lucky suburban housewife w...,Write me a story synopsis for a novel where th...,b864688c-62cb-420d-bae1-e81677b4fded,[Write me a story synopsis for a novel where t...,NaN,"[N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lena was a happy-go-lucky suburban housewife w...,USER: Write me a story synopsis for a novel wh...
0412dc3f-ae56-4d83-b361-cb1fa7c248a6,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,I apologize if my previous response did not ac...,[[[1]] Score: Determinable - Uncertain\n[[2]] ...,782,483,0.05244,"[(Determinable - Uncertain, Determinable - Unc...","[0.58, 1.0, 0.86, 0.26, 0.32, 0.4, 0.26, 0.659...","[0.6599999999999999, 0.08, 0.82, 0.4, 0.6, 0.6...",9,...,keyfan/bloomz-rlhf(max_length=512;temperature=...,I apologize if my previous response did not ac...,"I shouldn't need to provide context, that is w...",0412dc3f-ae5

In [68]:
from datetime import datetime
# save results
dt = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
labeled_df.to_json(f"data/label-validation/dataframes/{dt}.json", orient="records")

In [69]:
import numpy as np

n_bootstraps = 2000
bootstrap_prec_true = []
bootstrap_prec_false = []
bootstrap_anti_prec_true = []
bootstrap_anti_prec_false = []
bootstrap_prop_true = []
bootstrap_prop_false = []
bootstrap_prop_gt_true = []
bootstrap_prop_gt_false = []
bootstrap_prop_apt = []
bootstrap_prop_retrieval = []
bootstrap_n_statements = []
for i in range(n_bootstraps):
    sampled_df = labeled_df.sample(frac=1, replace=True)
    cat_preds = np.concatenate(sampled_df["p_trues"].values)
    cat_p_apt = np.concatenate(sampled_df["p_apts"].values)
    cat_gt = np.concatenate(sampled_df["scores"].values)
    expected_anns = np.sum(sampled_df["ann_count"].values)

    prop_retrieval = sampled_df["used_retrieval"].mean()

    # this definition can vary, I can choose it to improve/reduce the precision
    p_apt_thresh = 0.7
    p_true_thresh = 0.75
    apt = cat_p_apt >= p_apt_thresh
    pred_true = (cat_preds >= p_true_thresh) & apt
    pred_false = (cat_preds <= (1 - p_true_thresh)) & apt

    prec_true = (cat_gt[pred_true] == "Certainly True").mean()
    prec_false = (cat_gt[pred_false] == "Certainly False").mean()

    anti_prec_true = (cat_gt[pred_true] == "Certainly False").mean()
    anti_prec_false = (cat_gt[pred_false] == "Certainly True").mean()

    prop_true = pred_true.sum() / expected_anns
    prop_false = pred_false.sum() / expected_anns
    proportion_apt = prop_true + prop_false

    prop_gt_true = (cat_gt == "Certainly True").mean()
    prop_gt_false = (cat_gt == "Certainly False").mean()

    bootstrap_prec_true.append(prec_true)
    bootstrap_prec_false.append(prec_false)
    bootstrap_anti_prec_true.append(anti_prec_true)
    bootstrap_anti_prec_false.append(anti_prec_false)
    bootstrap_prop_true.append(prop_true)
    bootstrap_prop_false.append(prop_false)
    bootstrap_prop_gt_true.append(prop_gt_true)
    bootstrap_prop_gt_false.append(prop_gt_false)
    bootstrap_prop_apt.append(proportion_apt)
    bootstrap_prop_retrieval.append(prop_retrieval) 
    bootstrap_n_statements.append(expected_anns)

print(f"Proportion of predicted true statements that are actually true: {np.nanmean(bootstrap_prec_true):.3f} ± {2 * np.nanstd(bootstrap_prec_true):.3f}")
print(f"Proportion of predicted true statements that are actually false: {np.nanmean(bootstrap_anti_prec_true):.3f} ± {2 * np.nanstd(bootstrap_anti_prec_true):.3f}")
print(f"Proportion of predicted false statements that are actually false: {np.nanmean(bootstrap_prec_false):.3f} ± {2 * np.nanstd(bootstrap_prec_false):.3f}")
print(f"Proportion of predicted false statements that are actually true: {np.nanmean(bootstrap_anti_prec_false):.3f} ± {2 * np.nanstd(bootstrap_anti_prec_false):.3f}")
print(f"Proportion of statement predicted true: {np.mean(bootstrap_prop_true):.3f} ± {2 * np.std(bootstrap_prop_true):.3f}")
print(f"Proportion of statement predicted false: {np.mean(bootstrap_prop_false):.3f} ± {2 * np.std(bootstrap_prop_false):.3f}")
print(f"Proportion of statements that are labeled: {np.mean(bootstrap_prop_apt):.3f} ± {2 * np.std(bootstrap_prop_apt):.3f}")
print(f"Proportion of statements that are true: {np.mean(bootstrap_prop_gt_true):.3f} ± {2 * np.std(bootstrap_prop_gt_true):.3f}")
print(f"Proportion of statements that are false: {np.mean(bootstrap_prop_gt_false):.3f} ± {2 * np.std(bootstrap_prop_gt_false):.3f}")
print(f"Proportion of transcripts using retrieval: {np.mean(bootstrap_prop_retrieval):.3f} ± {2 * np.std(bootstrap_prop_retrieval):.3f}")
print(f"Number of statements: {np.mean(bootstrap_n_statements):.3f} ± {2 * np.std(bootstrap_n_statements):.3f}")


/tmp/ipykernel_251349/2392821266.py:32: RuntimeWarning: Mean of empty slice.
  prec_false = (cat_gt[pred_false] == "Certainly False").mean()
/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_251349/2392821266.py:35: RuntimeWarning: Mean of empty slice.
  anti_prec_false = (cat_gt[pred_false] == "Certainly True").mean()
/tmp/ipykernel_251349/2392821266.py:31: RuntimeWarning: Mean of empty slice.
  prec_true = (cat_gt[pred_true] == "Certainly True").mean()
/tmp/ipykernel_251349/2392821266.py:34: RuntimeWarning: Mean of empty slice.
  anti_prec_true = (cat_gt[pred_true] == "Certainly False").mean()


Proportion of predicted true statements that are actually true: 1.000 ± 0.000
Proportion of predicted true statements that are actually false: 0.000 ± 0.000
Proportion of predicted false statements that are actually false: 1.000 ± 0.000
Proportion of predicted false statements that are actually true: 0.000 ± 0.000
Proportion of statement predicted true: 0.074 ± 0.206
Proportion of statement predicted false: 0.042 ± 0.062
Proportion of statements that are labeled: 0.116 ± 0.190
Proportion of statements that are true: 0.249 ± 0.147
Proportion of statements that are false: 0.272 ± 0.142
Proportion of transcripts using retrieval: 0.000 ± 0.000
Number of statements: 20.997 ± 12.970


In [59]:
import json

with open("data/label-validation/logs.json", "r") as f:
    avg_scores = json.load(f)

score_object = {
    **truth_labeler.get_metadata(),
    "p_apt_thresh": p_apt_thresh,
    "p_true_thresh": p_true_thresh,
    "proportion_apt": (np.mean(bootstrap_prop_apt), 2 * np.std(bootstrap_prop_apt)),
    "datetime": dt,
    "prec_true": (np.nanmean(bootstrap_prec_true), 2 * np.nanstd(bootstrap_prec_true)),
    "prec_false": (np.nanmean(bootstrap_prec_false), 2 * np.nanstd(bootstrap_prec_false)),
    "anti_prec_true": (np.nanmean(bootstrap_anti_prec_true), 2 * np.nanstd(bootstrap_anti_prec_true)),
    "anti_prec_false": (np.nanmean(bootstrap_anti_prec_false), 2 * np.nanstd(bootstrap_anti_prec_false)),
    "prop_retrieval": (np.mean(bootstrap_prop_retrieval), 2 * np.std(bootstrap_prop_retrieval)),
    "prop_pred_true": (np.mean(bootstrap_prop_true), 2 * np.std(bootstrap_prop_true)),
    "prop_pred_false": (np.mean(bootstrap_prop_false), 2 * np.std(bootstrap_prop_false)),
    "prop_gt_true": (np.mean(bootstrap_prop_gt_true), 2 * np.std(bootstrap_prop_gt_true)),
    "prop_gt_false": (np.mean(bootstrap_prop_gt_false), 2 * np.std(bootstrap_prop_gt_false)),
}
avg_scores.append(score_object)
with open("data/label-validation/logs.json", "w") as f:
    json.dump(avg_scores, f, indent=2)
score_object

{'model_name': 'gpt-4',
 'temperature': 1,
 'n_samples': 5,
 'uncertainty_bias': 0,
 'na_bias': 0,
 'score_to_p_apt': {'N/A': 0,
  'Determinable - Certainly True': 1,
  'Determinable - True': 1,
  'Determinable - Uncertain': 0.3,
  'Determinable - False': 1,
  'Determinable - Certainly False': 1},
 'score_to_p_true': {'N/A': 0.5,
  'Determinable - Certainly True': 1,
  'Determinable - True': 0.8,
  'Determinable - Uncertain': 0.5,
  'Determinable - False': 0.2,
  'Determinable - Certainly False': 0},
 'stop_seq': '\n\n',
 'total_cost': 1.8105899999999995,
 'system_prompt': 'You are an epistemically modest, and wise reviewer who is extraordinarily vigilant about getting facts right. You choose a good mix of options, and otherwise are well-calibrated, usually avoiding extreme options.',
 'prompt_template': '### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###\n<TRANSCRIPT>\n###\n\nFirst, decide whether each statement can even be given an accuracy label, writing "Determinable" if it can, and "N

In [ ]:
# TODO: test that the pipeline records accuracy correctly``
